In [1]:
import re
import csv
import pandas as pd
import os
from camel_tools.utils import charsets
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('/Users/nurpeiis/Desktop/Capstone/hierarchical-did/utils')
from data_process import DataProcess

In [6]:
import xml.etree.ElementTree as et 
#Step 0: From xml to csv
xtree = et.parse('../../data_raw/ldc_ara_dialect_eng_para_2012_t09/data/BBN-Dialect_Arabic-English-Web.xml')
xroot = xtree.getroot()
rows_eg = []
rows_levant = []
cols = ["original_sentence", "dialect_region_id", "dialect_country_id"]
dialects = set()
for node in xroot: 
    if(node.find("DIALECT").text == 'EGYPTIAN'):
        rows_eg.append({'dialect_region_id': 'nile_basin', 'dialect_country_id': 'eg', 'original_sentence': node.find("SOURCE").text})
    else:
        rows_levant.append({'dialect_region_id': 'levant', 'dialect_country_id': '', 'original_sentence': node.find("SOURCE").text})
df_eg = pd.DataFrame(rows_eg, columns=cols)
df_eg.to_csv('../../data_raw/ldc_ara_dialect_eng_para_2012_t09/eg_processed.tsv', sep='\t', index=False)
df_levant = pd.DataFrame(rows_levant, columns=cols)
df_levant.to_csv('../../data_raw/ldc_ara_dialect_eng_para_2012_t09/levant_processed.tsv', sep='\t', index=False)


df_levant

,original_sentence,dialect_region_id,dialect_country_id
0,وهي مبروك مرة تانية,levant,
1,ناطرين مواضيعك بس هل الأيام وقت امتحانات رح نن...,levant,
2,ناولني المايك من عندك ... حاجتك صارلك ساعتين م...,levant,
3,بقا دعولي الله لا يضيعلي تعب و ينولني يلي ببالي,levant,
4,انتي جيبي معك سجاده صلاتك وبرأيي انو الصلاه ما...,levant,
...,...,...,...
138005,يسلمو نارووو و يسلمو نورووو « جاري التحميل »,levant,
138006,اما الباقي الوبر بيحميه,levant,
138007,بيضربوه سطعشر الف منية,levant,
138008,ما بتهيألى إنه فى شى بسر علشان نضحك بمعنى الضح...,levant,


In [10]:
dp = DataProcess('../data_processed_splited/ldc_ara_dialect_eng_para_2012_t09/', 'mechanical_turk_annotators', 'web_mixed', 'https://catalog.ldc.upenn.edu/LDC2012T09', 'ldc_ara_dialect_eng_para_2012_t09', {},{},0, 'corpus', 'manual')

In [11]:
dp.save_file('eg_processed.tsv', dp.preprocess('../../data_raw/ldc_ara_dialect_eng_para_2012_t09/eg_processed.tsv', '', '', 2, 1, header=0))
dp.save_file('levant_processed.tsv', dp.preprocess('../../data_raw/ldc_ara_dialect_eng_para_2012_t09/levant_processed.tsv', '', '', 2, 1, header=0))

In [8]:
dp.save_features('../datasets_splited_features.tsv')

/Users/nurpeiis/miniconda3/envs/hierarchical-did/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [12]:
files = ['eg_processed.tsv', 'levant_processed.tsv']
for file in files:
    df_train, df_dev, df_test = dp.split(file, 0.8, 0.10, 0.10)
    dp.save_file('train_'+file, df_train)    
    dp.save_file('dev_'+file, df_dev)
    dp.save_file('test_'+file, df_test)